In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import re
import time
import os
from pymongo import MongoClient
import pprint

%reload_ext dotenv
%dotenv

EMAIL = os.environ.get('USERNAME')
PASSWORD = os.environ.get('PASSWORD')
WEBSITE = os.environ.get('WEBSITE')

pp = pprint.PrettyPrinter(indent=4)

python_options = {'suffix':'.py',
                  'search_term' : 'python3',
                    'path': '/Volumes/Transcend/programs/prog1/python/'}
cpp_options = {'suffix':'.cpp',
               'search_term' : 'cpp',
                  'path': '/Volumes/Transcend/programs/prog1/cpp/'}

lang_options = python_options

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

options = Options()

# Headless option
options.add_argument("--headless")
options.add_argument("--window-size=1920x1080")

In [2]:
# Open up website
driver = webdriver.Chrome(chromedriver,options=options)
driver.get(WEBSITE)

In [3]:
# Navigate to Algorithms page
login_button = driver.find_elements_by_link_text('Login')[0]
login_button.click()
login_button2 = driver.find_elements_by_link_text('Login')[0]
login_button2.click()

username_field = driver.find_element_by_id('input-1')
username_field.send_keys(EMAIL) 

pw_field = driver.find_element_by_id('input-2')
pw_field.send_keys(PASSWORD) 


time.sleep(5)
driver.find_elements_by_tag_name('button')[0].click()
time.sleep(5)
driver.find_elements_by_tag_name('button')[0].click()

time.sleep(5)

driver.find_element_by_link_text('Algorithms').click()

In [4]:
# Scroll down page to find challenge of choice

SCROLL_PAUSE_TIME = 1.0
req_challenge_id = 35
# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    challenge_list = driver.find_element_by_class_name('challenges-list')
    num_challenges_on_page = len(challenge_list.find_elements_by_class_name('challenge-list-item'))


    if new_height == last_height or num_challenges_on_page > req_challenge_id:
        break
    
    last_height = new_height


In [5]:
# Get individual challenge data and click 
time.sleep(1)
challenge_list = driver.find_element_by_class_name('challenges-list')
challenge_item = challenge_list.find_elements_by_class_name('challenge-list-item')[0]

challenge_title = challenge_item.find_element_by_class_name('challengecard-title').text.split('\n')[0]
challenge_difficulty = challenge_item.find_element_by_class_name('difficulty').text
max_score_str = challenge_item.find_element_by_class_name('max-score').text
max_score= float(re.findall('\d+', max_score_str)[0])
success_rate_str = challenge_item.find_element_by_class_name('success-ratio').text
success_rate = 0.01*float(re.findall('\\d+(?:\\.\\d+)?', success_rate_str)[0])
print(f'challenge_title:{challenge_title}')
print(f'challenge_difficulty:{challenge_difficulty}')
print(f'max_score:{max_score}')
print(f'success_rate:{success_rate}')

challenge_item.click()

challenge_title:Simple Array Sum
challenge_difficulty:Easy
max_score:10.0
success_rate:0.9454000000000001


In [6]:
# Go to Leaderboard and filter by language
time.sleep(1)
driver.find_element_by_link_text('Leaderboard').click()

reveal_button = driver.find_elements_by_tag_name('button')
try:
    reveal_button[0].click()
    driver.find_element_by_class_name('hr_primary-btn').click()
except:
    print('solutions already revealed')
    
time.sleep(1)
language_field = driver.find_elements_by_class_name('ac-input')[1]
time.sleep(1)
language_field.send_keys(lang_options['search_term'])

time.sleep(5)
language_field.send_keys(Keys.ENTER)
time.sleep(5)

solutions already revealed


In [7]:
# Connect to database
client = MongoClient()
db = client.py2cpp
py = db.python
cpp = db.cpp

In [8]:

# Collect solutions
max_page_number = 100
solutions_per_page = 20 # Do not change. This is the default by site.

# Loops through pages
for i in range(max_page_number):
    # Loops through solutions on page
    for j in range(solutions_per_page):
        try:
            # Get meta data on solution such as rank, language, and score
            row = driver.find_elements_by_class_name('table-row')[j]
            elementList = row.find_elements_by_class_name('table-row-column')
            text_info = [ elem.text for elem in elementList ]
            rank = int(text_info[1])
            language = text_info[3]
            score = float(text_info[4])
            
            # Navigate to solution page and get code as a string
            solution_links = driver.find_elements_by_link_text('View solution')
            lin = solution_links[j]
            url = lin.get_attribute('href')
            driver.get(url)
            b = driver.find_element_by_tag_name('body')
            code = b.text

            # Save to mongodb server
            doc = {
                'challenge_title': challenge_title,
                'challenge_difficulty': challenge_difficulty,
                'max_score': max_score,
                'success_rate':success_rate,
                'rank' : rank,
                'language' : language,
                'score': score,
                'code' : code
            }
            pp.pprint(doc)
            py.insert_one(doc)

            # Navigate back to solutions list
            driver.back()
            time.sleep(0.25*np.random.random())
            
        except:
            pass
        
    # Navigate to next page if it exists
    time.sleep(3+np.random.random())    
    next_page_elem = driver.find_element_by_class_name('next-page')
    if 'disabled' in next_page_elem.get_attribute('class'):
        break
    else:
        next_page_elem.click()
        
        

{   'challenge_difficulty': 'Easy',
    'challenge_title': 'Simple Array Sum',
    'code': 'n = int(input())\n'
            'nums = list(map(int, input().split()))\n'
            'sum = 0\n'
            'for num in nums:\n'
            '    sum += num\n'
            'print(sum)',
    'language': 'Python 3',
    'max_score': 10.0,
    'rank': 1,
    'score': 10.0,
    'success_rate': 0.9454000000000001}
{   'challenge_difficulty': 'Easy',
    'challenge_title': 'Simple Array Sum',
    'code': 'n = input()\n'
            "l = list(map(int, input().split(' ')))\n"
            'print(sum(l))',
    'language': 'Python 3',
    'max_score': 10.0,
    'rank': 1,
    'score': 10.0,
    'success_rate': 0.9454000000000001}
{   'challenge_difficulty': 'Easy',
    'challenge_title': 'Simple Array Sum',
    'code': 'import sys \n'
            '\n'
            'n = int(input())\n'
            'data = sys.stdin.readlines()[0].split()\n'
            '\n'
            's = 0 \n'
            '\n'
        

{   'challenge_difficulty': 'Easy',
    'challenge_title': 'Simple Array Sum',
    'code': 'n=int(input())\n'
            'l=[int(i) for i in input().split()]\n'
            'print(sum(l))',
    'language': 'Python 3',
    'max_score': 10.0,
    'rank': 1,
    'score': 10.0,
    'success_rate': 0.9454000000000001}
{   'challenge_difficulty': 'Easy',
    'challenge_title': 'Simple Array Sum',
    'code': '',
    'language': 'Python 3',
    'max_score': 10.0,
    'rank': 1,
    'score': 10.0,
    'success_rate': 0.9454000000000001}
{   'challenge_difficulty': 'Easy',
    'challenge_title': 'Simple Array Sum',
    'code': 'int(input())\n'
            'nums = input().split()\n'
            'nums = list(map(int, nums))\n'
            'sum = 0\n'
            '\n'
            '\n'
            'for num in nums:\n'
            '    sum += num\n'
            'print(sum)',
    'language': 'Python 3',
    'max_score': 10.0,
    'rank': 1,
    'score': 10.0,
    'success_rate': 0.9454000000000001}


{   'challenge_difficulty': 'Easy',
    'challenge_title': 'Simple Array Sum',
    'code': 'Num = int(input())\n'
            'Num2 = input().split(" ")\n'
            'lst = list()\n'
            'for i in Num2:\n'
            '    i = int(i)\n'
            '    lst.append(i)\n'
            '    sums = sum(lst)\n'
            'print(sums)',
    'language': 'Python 3',
    'max_score': 10.0,
    'rank': 1,
    'score': 10.0,
    'success_rate': 0.9454000000000001}
{   'challenge_difficulty': 'Easy',
    'challenge_title': 'Simple Array Sum',
    'code': 'N = int(input())\n'
            '\n'
            'array = [int(x) for x in input().split()]\n'
            'print(sum(array[:N]))',
    'language': 'Python 3',
    'max_score': 10.0,
    'rank': 1,
    'score': 10.0,
    'success_rate': 0.9454000000000001}
{   'challenge_difficulty': 'Easy',
    'challenge_title': 'Simple Array Sum',
    'code': 'x = int(input())\n'
            'y = input().split()\n'
            'sum = 0\n'
          

{   'challenge_difficulty': 'Easy',
    'challenge_title': 'Simple Array Sum',
    'code': 'sice = int(input())\n'
            'xs = list(map(int,input().split()))\n'
            'sum = 0\n'
            'for x in xs:\n'
            '    sum = sum+x\n'
            'print(sum)',
    'language': 'Python 3',
    'max_score': 10.0,
    'rank': 1,
    'score': 10.0,
    'success_rate': 0.9454000000000001}
{   'challenge_difficulty': 'Easy',
    'challenge_title': 'Simple Array Sum',
    'code': 'count = int(input())\n'
            '\n'
            'sum = 0\n'
            "for i in input().split(' '):\n"
            '    sum += int(i)\n'
            '    \n'
            'print(sum)',
    'language': 'Python 3',
    'max_score': 10.0,
    'rank': 1,
    'score': 10.0,
    'success_rate': 0.9454000000000001}
{   'challenge_difficulty': 'Easy',
    'challenge_title': 'Simple Array Sum',
    'code': 'import sys\n'
            '\n'
            'N = int(sys.stdin.readline())\n'
            'elts = 

KeyboardInterrupt: 

In [10]:
# See docs in python collection
len(list(py.find({})))

80